# Big Data Modeling and Management Assigment - Homework 1

## 🍺 The Beer project  🍺 

Graph databases are a natural way of navigating distinct types of data. For this first project we will be taking a graph database to analyse beer and breweries!   

_For reference the dataset used for this project has been extracted from [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. Even though the author does not present metadata on the origin of the data it is probably a collection of open data from places like [beeradvocate](https://www.beeradvocate.com/)_.

### Problem description

Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions with python syntax. Submit the results by following the instructions.


### Questions

1. How many distinct countries exist in the database? [Hint: mind repetitions]
1. Most reviews:  
    1. Which `Beer` has the most reviews?  
    1. Which `Brewery` has the most reviews for its beers? [Hint: 3-node path]
    1. Which `Country` has the most reviews for its beers? [Hint: 5-node path]
1. Who is/are the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?
1. Which Portuguese brewery has the most beers?
1. From those beers (the ones returned from the previous question), which has the most reviews?
1. On average how many different beer styles does each brewery produce?
1. Which brewery produces the strongest beers according to ABV? [Hint: database has NaN values]
1. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try? (Justify your answer) [Hint: database has NaN values]
1. Using Graph Algorithms answer **one** of the following questions: [NB: make sure to clear the graph before using it again]
    1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    2. Which beer is the most influential when considering the number of users who reviewed it? [Please use limit of 1000 on beer-review-user path]]
    3. Users are connected together by their reviews of beers, taking into consideration the "overall" score they assign as a weight, how many communities are formed from these relationships? How many users are in the largest community? [Please use limit of 1000 on beer-review-user path]]
1. Using Graph Algorithms answer **one** of the following questions:
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`? [Hint:inspect two subsets: with and without the beer in question]
    2. Which user is the most influential when it comes to reviews made?
1. If you had to pick 3 beers to recommend using only this database, which would you pick and why? (Justify your answer) [Hint: database has NaN values]


Questions 8 to 10 are somewhat open, which means we'll also be evaluating the reasoning behind your answer. So there aren't necessarily bad results there are only wrong criteria, explanations, or execution. 
 
### Groups  

Groups should have 4 to 5 people.
You should register your group on **moodle**.

### Submission      

The code used to produce the results and respective explanations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself (please submit one Jupyter notebook per group).

### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and simplicity of the solution.  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

## Loading the Database

#### Be sure that you **don't have** the neo4j docker container from the classes running (you can Stop it in the desktop app or with the command "`docker stop Neo4JLab`")


The default container does not have any data whatsoever, we will have to load a database into our docker image:
- Download and unzip the `Neo4JHWData` file provided on Moodle (apr. 6 Gb).
- Copy the path of the `Neo4JHWData` folder of the unziped file, e.g. `c:\Users\osavc\Documents\Nova_BDMM\_2024\Homework\Neo4JHWData/data`.
- Download and unzip the `Neo4JPlugins` file provided on Moodle (it is the same plugins as for the labs).
- Copy the path of the `Neo4JPlugins` folder of the unziped file, e.g. `c:\Users\osavc\Documents\Nova_BDMM\_2024\Installations\Neo4JPlugins`.
- Change the code below to include your correct folder paths. Be sure that you have Docker running first:

`docker run --name Neo4JHW -p 7474:7474 -p 7687:7687 -d -v "C:\Users\osavc\Documents\Nova_BDMM\_2024\Installations\Neo4JPlugins":/plugins -v "C:\Users\osavc\Documents\Nova_BDMM\_2024\HW1\Neo4JHWData/data":/data --env NEO4J_AUTH=neo4j/test --env NEO4J_dbms_connector_https_advertised__address="localhost:7473" --env NEO4J_dbms_connector_http_advertised__address="localhost:7474" --env NEO4J_dbms_connector_bolt_advertised__address="localhost:7687" --env NEO4J_dbms_security_procedures_unrestricted="gds.*" --env NEO4J_dbms_security_procedures_allowlist="gds.*" neo4j:4.4.5`

- Since Neo4j is trying to recognize a new database folder, this might take a bit  before you can view the data on Neo4j webpage (let's say 3 minutes).

In [1]:
from neo4j import GraphDatabase
from pprint import pprint

In [2]:
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="test"

In [3]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), )

### All The Functions you'll need to run queries in Neo4J

In [4]:
def execute_read(driver, query):    
    with driver.session(database="neo4j") as session:
        result = session.execute_read(lambda tx, query: list(tx.run(query)), query)
    return result

### Understanding the Database

In [5]:
query = """
        call db.labels();
    """

result = execute_read(driver, query)

pprint(result)

[<Record label='COUNTRIES'>,
 <Record label='CITIES'>,
 <Record label='BREWERIES'>,
 <Record label='BEERS'>,
 <Record label='REVIEWS'>,
 <Record label='STYLE'>,
 <Record label='USER'>]


In [6]:
query = """
        CALL db.relationshipTypes();
    """

result = execute_read(driver, query)

pprint(result)

[<Record relationshipType='REVIEWED'>,
 <Record relationshipType='BREWED'>,
 <Record relationshipType='IN'>,
 <Record relationshipType='HAS_STYLE'>,
 <Record relationshipType='POSTED'>]


# Submission

GROUP NUMBER:

**XXXXXX**

GROUP MEMBERS:

|STUDENT NUMBER|STUDENT NAME|
|---|---|
|20230974|Devora Cavaleiro|
|20230520|Guilherme Sá|
|20230739|Maria Batrakova|
|20230372|Sebastião Rosalino|
|20230548|Vitor Souto|

#### 1. How many distinct countries exist in the database?

In [7]:
# Define a Cypher query to count distinct country names in the database (minding repititions)
# The DISTINCT c.name is to avoid counting duplicate countries and to only count the unique country names encounterd
# No relationship is used in this query, only distinct country names are counted

query = """
        MATCH (c:COUNTRIES)
        RETURN COUNT(DISTINCT c.name)
        """

# Execute the Cypher query using the provided `execute_read` function
# The query result is a list where each item is a record object containing the query results
result = execute_read(driver, query)

# Extract the first row and column from the result set, which contains the count of distinct countries (by their names)
# `result[0][0]` is because of the fact that the function `execute_read` returns a list of lists
# where it is possible to get the count by indexing into the first element
print(f'The number of distinct countries in the database is: {result[0][0]}')

The number of distinct countries in the database is: 200


#### 2. Most reviews:  
    A) Which `Beer` has the most reviews?  
    B) Which `Brewery` has the most reviews for its beers?
    C) Which `Country` has the most reviews for its beers? 

*Which `Beer` has the most reviews?*

In [8]:
# This query finds the most reviewed beer
# Match nodes with label 'BEERS' with relationships of type ':REVIEWED', leading to nodes labeled 'REVIEWS'

query = """
        MATCH (b:BEERS)-[:REVIEWED]->(r:REVIEWS)
        RETURN b.name AS Beer_Name, COUNT(r) AS Number_of_Reviews 
        ORDER BY Number_of_Reviews DESC
        LIMIT 1
        """

# Execute the Cypher query using the function 'execute_read'
# The result is stored in the variable 'result'
result = execute_read(driver, query)

# Printing the result
# 'result[0]["Beer_Name"]','result[0]["Number_of_Reviews"]' accesses the first row and the corresponding column of the
# result set, which contains the count of the number of reviews and the name of the beer
print(f'"{result[0]["Beer_Name"]}" is the beer with the most reviews, being reviewed {result[0]["Number_of_Reviews"]} times')

"IPA" is the beer with the most reviews, being reviewed 31387 times


*Which `Brewery` has the most reviews for its beers?*

In [9]:
# This query finds the brewery with the most reviews for its beers
#  - Starts with nodes labeled 'BREWERIES'.
#  - Find relationships of type ':BREWED' connecting them to nodes labeled 'BEERS'
#  - Then, find relationships of type ':REVIEWED' connecting the beers to nodes labeled 'REVIEWS'
# Finally, returns the name of the brewery with the most reviews, counted in 'Number_Of_Reviews'.

query = """
        MATCH (br:BREWERIES)-[:BREWED]->(b:BEERS)-[:REVIEWED]->(r:REVIEWS)
        RETURN br.name AS Brewery_Name, COUNT(r) AS Number_Of_Reviews
        ORDER BY Number_Of_Reviews DESC
        LIMIT 1
        """

# Execute the Cypher query using the function 'execute_read'
# The result is stored in the variable 'result'
result = execute_read(driver, query)

# Printing the result
# 'result[0]["Brewery_Name"]','result[0]["Number_of_Reviews"]' accesses the first row and the corresponding column of the
# result set, which contains the count of the number of reviews and the name of the brewery
print(f'"{result[0]["Brewery_Name"]}" is the Brewery with the most reviews for its beers, with a total number of reviews of: {result[0]["Number_Of_Reviews"]}')

"Sierra Nevada Brewing Co." is the Brewery with the most reviews for its beers, with a total number of reviews of: 175161


*Which `Country` has the most reviews for its beers?*

In [10]:
# This query finds the country with the most reviews for its beers
# - Starts with nodes labeled 'COUNTRIES'.
# - Use relationships of type ':IN' to find connected nodes labeled 'CITIES'
# - Follow another ':IN' relationship to reach nodes labeled 'BREWERIES' within those cities
# - Then, find relationships of type ':BREWED' connecting breweries to nodes labeled 'BEERS'
# - Finally, find relationships of type ':REVIEWED' connecting the beers to nodes labeled 'REVIEWS'
# Finally, returns the name of the country and counts the reviews for beers from that country

query = """
        MATCH (cnt:COUNTRIES)<-[:IN]-(c:CITIES)<-[:IN]-(br:BREWERIES)-[:BREWED]->(b:BEERS)-[:REVIEWED]->(r:REVIEWS)
        RETURN cnt.name AS Country_Name, COUNT(r) AS Number_Of_Reviews
        ORDER BY Number_Of_Reviews DESC
        LIMIT 1
        """

# Execute the Cypher query using the function 'execute_read'
# The result is stored in the variable 'result'
result = execute_read(driver, query)

# Printing the result
# 'result[0]["Country_Name"]','result[0]["Number_of_Reviews"]' accesses the first row and the corresponding column of the
# result set, which contains the count of the number of reviews and the country
print(f'{result[0]["Country_Name"]} is the country with the most reviews for its beers, with a total number of reviews of: {result[0]["Number_Of_Reviews"]}')

US is the country with the most reviews for its beers, with a total number of reviews of: 7675804


#### 3. Who is/are the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?

In [11]:
# This query finds the user with the most shared reviews with the user 'CTJman'
# - Starts with a node labeled 'USER', having a relationship of type ':POSTED' to reviews that the user posted
# - Then, follows a ':REVIEWED' relationship to a beer node being reviewed
# - Then, another review for the same beer, which is connected to another review node
# - Finally, a ':POSTED' relationship from that second review node to a specific user node
# Only reviews posted by a user (u) about a beer that was also reviewed by CTJman will be matched
# It returns the user's name, and counts the number of reviews shared

query = '''
    MATCH (u:USER)-[:POSTED]-(r:REVIEWS)-[:REVIEWED]-(b:BEERS)-[:REVIEWED]-(:REVIEWS)-[:POSTED]-(:USER{name:'CTJman'})
    RETURN u.name AS User_Name, COUNT(r) AS Number_Of_Shared_Reviews
    ORDER BY COUNT(r) DESC
    LIMIT 1
        '''

# Execute the Cypher query using the function 'execute_read'
# The result is stored in the variable 'result'
result = execute_read(driver, query)

# Printing the result
# 'result[0]["User_Name"]','result[0]["Number_Of_Shared_Reviews"]' accesses the first row and the corresponding column of the
# result set, which contains the count of the number of shared reviews and the user name
print(f'The user with most shared reviews of the same beers with user CTJman is: "{result[0]["User_Name"]}", with {result[0]["Number_Of_Shared_Reviews"]} shared reviews.')

The user with most shared reviews of the same beers with user CTJman is: "acurtis", with 1428 shared reviews.


#### 4. Which Portuguese brewery has the most beers?


In [12]:
# This query finds the Portuguese brewery with the most distinct beers
# - Starts with a country node with the name 'PT' (Portugal)
# - Use relationships of type ':IN' to find connected nodes 'CITIES'
# - Follows another ':IN' relationship to reach nodes labeled 'BREWERIES' within those cities
# - Then, find relationships of type ':BREWED' connecting breweries to nodes labeled 'BEERS'
# Country with the name 'PT' acts as a filter, ensuring only breweries in Portugal are considered
# It returns the name of the brewery, and the count of the number of unique beers
query = """
        MATCH (:COUNTRIES {name:'PT'})<-[:IN]-(c:CITIES)<-[:IN]-(br:BREWERIES)-[:BREWED]->(b:BEERS)
        RETURN br.name AS Brewery_Name, COUNT(DISTINCT b.name) AS Beer_Count
        ORDER BY Beer_Count DESC
        LIMIT 1
        """

# Execute the Cypher query using the function 'execute_read'
# The result is stored in the variable 'result'
result = execute_read(driver, query)

# Printing the result
# 'result[0]["Brewery_Name"]','result[0]["Beer_Count"]' accesses the first row and the corresponding column of the
# result set, which contains the count of the number of beers and the brewery's name
print(f'"{result[0]["Brewery_Name"]}" is the Portuguese Brewery that brews the most beers ({result[0]["Beer_Count"]})')

"Dois Corvos Cervejeira" is the Portuguese Brewery that brews the most beers (40)


#### 5. From those beers (the ones produced in the brewery from the previous question), which has the most reviews?


In [13]:
# This query finds the most reviewed beer brewed by Dois Corvos Cervejeira
# - Starts with a brewery node with the name 'Dois Corvos Cervejeira'
# - Follow a relationship of type ':BREWED' to connect it to beers that the brewery brewed
# - Then, find relationships of type ':REVIEWED' connecting the beers to nodes labeled 'REVIEWS'
# A brewery with the name 'Dois Corvos Cervejeira' acts as a filter, ensuring only reviews for beers from that brewery
# are considered
# It returns the name of the beer, and counts the number of reviews
query = '''
    MATCH (:BREWERIES{name:'Dois Corvos Cervejeira'})-[:BREWED]-(b:BEERS)-[:REVIEWED]-(r:REVIEWS) 
    RETURN b.name AS Beer_Name, COUNT(r) AS Number_Of_Reviews
    ORDER BY Number_Of_Reviews DESC
    LIMIT 1
    '''

# Execute the Cypher query using the function 'execute_read'
# The result is stored in the variable 'result'
result = execute_read(driver, query)

# Printing the result
# 'result[0]["Beer_Name"]','result[0]["Number_Of_Reviews"]' accesses the first row and the corresponding column of the
# result set, which contains the count of the number of reviews and the beer's name
print(f'The beer from "Dois Corvos Cervejeira" with most reviews is: "{result[0]["Beer_Name"]}", with {result[0]["Number_Of_Reviews"]} reviews.')

The beer from "Dois Corvos Cervejeira" with most reviews is: "Finisterra", with 10 reviews.


#### 6. On average how many different beer styles does each brewery produce?

In [14]:
# This query finds the average number of distinct beer styles per brewery
# - Starts with nodes labeled 'BREWERIES'
# - Follow relationships of type ':BREWED' to connect them to beers they brewed
# - Then, follow relationships of type ':HAS_STYLE' connecting the beers to style nodes
# It returns the name of the brewery, and counts the distinct beer's styles
query = '''
    MATCH (br:BREWERIES)-[:BREWED]-(b:BEERS)-[:HAS_STYLE]-(s:STYLE) 
    WITH br.name AS Brewery_Name, COUNT(DISTINCT(s)) AS Styles 
    RETURN AVG(Styles)
        '''

# Execute the Cypher query using the function 'execute_read'
# The result is stored in the variable 'result'
result = execute_read(driver, query)

# Printing the result
# 'result[0][0]' accesses the first row and the first column of the result set, which contains the average style
print(f'The average different styles of beer per brewery is approximately (rounded to 2 decimal places): {result[0][0]:.2f}')

The average different styles of beer per brewery is approximately (rounded to 2 decimal places): 10.67


#### 7. Which brewery produces the strongest beers according to ABV?

In [15]:
# This query finds the brewery with the highest average ABV (alcohol by volume)
# - Starts with nodes labeled 'BREWERIES'
# - Follow relationships of type ':BREWED' to connect them to beers they brewed
# The WITH clause creates a temporary result set with two values for each beer:
#  - abv: The alcohol by volume (converted to a float using toFloat) of the beer
#  - brewery: The name of the brewery that brewed the beer (br.name)
# The WHERE clause filters the temporary result set. It keeps only entries where the beer's ABV is different
# from NaN, in an efficient and faster way, since comparing floats is faster than comparing strings. So, only breweries with beers having non-null ABV values are considered
# It returns the brewery's name and the average ABV, in descending order
query1 = '''
     MATCH (br:BREWERIES)-[:BREWED]-(b:BEERS) 
     WITH toFloat(b.abv) AS abv, br.name AS brewery 
     WHERE b.abv <> abv 
     RETURN brewery, AVG(abv) AS avg_abv 
     ORDER BY avg_abv DESC
     LIMIT 1
'''

# Execute the Cypher query using the function 'execute_read'
# The result is stored in the variable 'result1'
result1 = execute_read(driver, query1)


# This query finds the top 2 breweries (the top 2 was found empirically) that produce beers with the highest maximum ABV
# - Start with nodes labeled 'BREWERIES'
# - Follow relationships of type ':BREWED' to connect them to beers they brewed
# The WITH clause creates a temporary result set with two values for each beer:
#  - abv: The alcohol by volume (converted to a float using toFloat) of the beer
#  - brewery: The name of the brewery that brewed the beer (br.name)
# The WHERE clause keeps only entries where the beer's ABV (b.abv) is different from the NaN, in an efficient and faster way, since comparing floats is faster than comparing strings. 
# This ensures only breweries producing beers with non-null abv are considered
# It returns the 2 brewery's name, and the maximum ABV, in descending order
query2 = '''
    MATCH (br:BREWERIES)-[:BREWED]-(b:BEERS) 
    WITH toFloat(b.abv) AS abv, br.name AS brewery 
    WHERE b.abv <> abv 
    RETURN brewery, MAX(abv) AS max_abv 
    ORDER BY max_abv DESC 
    LIMIT 2
'''

# Execute the Cypher query using the function 'execute_read'
# The result is stored in the variable 'result2'
result2 = execute_read(driver, query2)


# Printing the results
# 'result1[0]["brewery"]','result1[0]["avg_abv"]' accesses the first row and the corresponding column of the
# result set, which contains the average ABV and the brewery's name

# 'result2[0]["max_abv"]','result2[0]["brewery"]','result2[1]["brewery"]' accesses the first row and the corresponding column of the
# result set, which contains the maximun ABV, the brewery's name, and in the last case, it accesses the second row and the corresponding column of the
# result set, which contains the other brewery's name
print(f'On average, "{result1[0]["brewery"]}" produces the strongest beers, with {result1[0]["avg_abv"]:.2f} average ABV.\n')

print(f'Two breweries produce at least one beer with the maximum ABV of {result2[0]["max_abv"]}. They are: "{result2[0]["brewery"]}" and "{result2[1]["brewery"]}"')

On average, "1648 Brewing Company Ltd" produces the strongest beers, with 25.58 average ABV.

Two breweries produce at least one beer with the maximum ABV of 100.0. They are: "Avondale Brewing Co." and "1648 Brewing Company Ltd"


#### 8. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?

In [16]:
# This query finds the beer style with the highest average review score (smell + look)
# - Start with nodes labeled 'STYLE'
# - Follow relationships of type ':HAS_STYLE' to connect them to beers with that style
# - Then, follow relationships of type ':REVIEWED' connecting the beers to nodes labeled 'REVIEWS'
# The WITH clause creates a temporary result set with three values for each review:
# - smell: The smell rating of the review (converted to a float using tofloat)
# - look: The look rating of the review (converted to a float using tofloat)
# - name: The name of the style associated with the reviewed beer (s.name)
# The WHERE clause keeps only entries where the review's smell rating (r.smell) and look rating (r.look) are different from NaN, in an efficient and fast way, since it is faster to 
# compare floats than strings
# It returns the style's name, and a score by averaging smell and look ratings, in descending order
query = '''
    MATCH (s:STYLE)-[:HAS_STYLE]-(:BEERS)-[:REVIEWED]-(r:REVIEWS) 
    WITH tofloat(r.smell) AS smell, tofloat(r.look) AS look, s.name AS name  
    WHERE smell <> r.smell AND look <> r.look 
    RETURN DISTINCT(name), (AVG(smell) + AVG(look))/2 AS score 
    ORDER BY score DESC
    LIMIT 1
    '''

# Execute the Cypher query using the function 'execute_read'
# The result is stored in the variable 'result'
result = execute_read(driver, query)

# Printing the result
# 'result[0]["name"]' accesses the first row and the corresponding column of the
# result set, which contains the style's name
print(f'If you typically enjoy a beer due to its aroma and appearance, we would suggest you try the beer style of: "{result[0]["name"]}"')

If you typically enjoy a beer due to its aroma and appearance, we would suggest you try the beer style of: "New England IPA"


**Justification**

To provide a recommendation for someone who typically enjoys a beer due to its aroma and appearance, the recommendation algorithm focuses on two key aspects from the reviews: the smell and the look ratings. These aspects are highly relevant to the sensory experience of beer, which directly aligns with the user's stated preferences.

Here's a justification for the approach taken by the recommendation algorithm:

**Focus on Relevant Factors**: Since the user's enjoyment is primarily driven by aroma (smell) and appearance (look), the algorithm selectively considers only these two factors from the reviews. This ensures that the recommendation is based on attributes that are most important to the user's experience.

**Use of Averaging:** By averaging the smell and look ratings across all reviews for beers of a particular style, the algorithm derives a composite score that represents the overall sensory appeal of each beer style. This aggregation method effectively summarizes the collective opinion of the beer community regarding these sensory characteristics.

**Handling Non-Numeric Values:** The algorithm includes a safeguard to ensure data quality by converting ratings to floats and excluding entries where the smell or look rating is not a number (NaN). This step prevents the algorithm from being skewed by invalid or missing data.

**Efficient Comparison:** By comparing the ratings as floats instead of strings, the algorithm enhances its performance. This is a crucial step, considering the potentially large volume of reviews that must be processed, should the database scale even further.

**Distinct Styles and Highest Score:** The algorithm returns the name of the beer style with the highest average score for the combined smell and look ratings. By focusing on all distinct styles, it ensures that the maximum diverse range of beer experiences is considered, and by ordering by the score, it identifies the style that best matches the user's preferences.

**Limiting Results:** The query limits the result to the top beer style to provide a clear and singular recommendation, making the decision process simpler for the user.

#### 9. Using Graph Algorithms answer **one** of the following questions:
    A. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    B. Which beer is the most influential when considering the number of users who reviewed it?
    C. Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users are in the largest community?

**C) Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users are in the largest community?**

In [17]:
# Step 0 - Try to drop an existing graph with the name 'user_review_community' to ensure that it is possible to create a new graph
try:
    query = """
            CALL gds.graph.drop('user_review_community') YIELD graphName;
        """
    # Execute the Cypher query to drop the graph
    result = execute_read(driver, query)
except Exception as e:
    # Catch and print any exceptions that occur during the drop operation
    pprint('Graph did not exist. Creating a new one')


# Step 1 - Project a new graph representing the user review community
try:
    # This query projects a graph for the user review community with specific conditions and limitations
    # Nodes in this graph are identified based on their labels ('USER' and 'BEERS') and their connections through 'REVIEWS'
    # Relationships in this projection are defined by a sequence of 'POSTED' and 'REVIEWED' relationships between 'USER' and 'BEERS'
    # The 'overall' score from 'REVIEWS' is used as the weight for these relationships
    # A limit of 1000 is applied to constrain the number of relationships processed, aiming to improve query performance
    query_projection = """
        CALL gds.graph.project.cypher(
            'user_review_community',
            
            // Node projection: 
            // Selects nodes based on the 'USER' label that have posted reviews
            'MATCH (n:USER)-[:POSTED]-(r:REVIEWS) RETURN id(n) AS id',
             
            // Relationship projection:
            // Constructs relationships between users based on shared reviews of beers, 
            // applying several conditions:
            // 1. Ensures that there are more than one review for a beer to be considered
            // 2. Matches users who have posted reviews with an equal 'overall' score for the same beer
            // 3. Filters pairs to ensure that only unique user pairs (u1, u2) are considered,
            //    based on the uniqueness of their internal IDs and the equality of review scores
            // The 'overall' score is used as the weight for these relationships

            'MATCH (r:REVIEWS)-[:REVIEWED]-(b:BEERS)
                WITH COUNT(r) AS n_reviews, b
                WHERE n_reviews > 1
                    MATCH (u1:USER)-[:POSTED]-(r1:REVIEWS)-[:REVIEWED]-(b)-[:REVIEWED]-(r2:REVIEWS)-[:POSTED]-(u2:USER)
                     WHERE toFloat(r1.overall) <> r1.overall AND 
                           toFloat(r1.overall) = toFloat(r2.overall) AND 
                           id(u1) < id(u2)
                     RETURN id(u1) AS source, id(u2) AS target, toFloat(r1.overall) AS weight
                LIMIT 1000'
        )
    """
    # Executes the query to create the graph projection and prints the result,
    # including details about the projected graph
    result = execute_read(driver, query_projection)
    pprint(result)
    print('\n')  # Adds a newline for better readability of the output
except Exception as e:
    # Catches and prints any exceptions that occur during the graph projection operation
    pprint(e)


# Step 2 - Compute communities within the projected graph
try:
    # Run the label propagation algorithm to detect communities
    # using the overall score as the weight for connections
    query_communities = """
        CALL gds.labelPropagation.stats('user_review_community', { relationshipWeightProperty: 'weight' })
        YIELD communityCount, ranIterations, didConverge
    """
    # Execute the Cypher query to detect communities
    communities_result = execute_read(driver, query_communities)
    # Pretty print the results, including the number of communities, iterations run, and convergence status
    pprint(f'The number of found communities is: {communities_result[0][0]}')
except Exception as e:
    # Catch and print any exceptions that occur during community detection
    pprint(e)


# Step 3 - Identify the largest community from the detected communities
try:
    # Run the label propagation algorithm to get individual community sizes
    # and identify the largest community by ordering the sizes in descending order
    query_largest_community = """
        CALL gds.labelPropagation.stream('user_review_community', { relationshipWeightProperty: 'weight' })
        YIELD nodeId, communityId AS Community
            
        WITH gds.util.asNode(nodeId).name AS Name, Community
        RETURN Community, count(*) as CommunitySize
        ORDER BY CommunitySize DESC
        LIMIT 1
    """
    # Execute the Cypher query to identify the largest community
    largest_community_result = execute_read(driver, query_largest_community)
    # Pretty print the result which includes the size of the largest community
    print('\n')
    pprint(f'The number of users in the largest community is: {largest_community_result[0][1]}')
except Exception as e:
    # Catch and print any exceptions that occur while identifying the largest community
    pprint(e)

'Graph did not exist. Creating a new one'
[<Record nodeQuery='MATCH (n:USER)-[:POSTED]-(r:REVIEWS) RETURN id(n) AS id' relationshipQuery='MATCH (r:REVIEWS)-[:REVIEWED]-(b:BEERS)\n                WITH COUNT(r) AS n_reviews, b\n                WHERE n_reviews > 1\n                    MATCH (u1:USER)-[:POSTED]-(r1:REVIEWS)-[:REVIEWED]-(b)-[:REVIEWED]-(r2:REVIEWS)-[:POSTED]-(u2:USER)\n                     WHERE toFloat(r1.overall) <> r1.overall AND \n                           toFloat(r1.overall) = toFloat(r2.overall) AND \n                           id(u1) < id(u2)\n                     RETURN id(u1) AS source, id(u2) AS target, toFloat(r1.overall) AS weight\n                LIMIT 1000' graphName='user_review_community' nodeCount=164935 relationshipCount=1000 projectMillis=19134>]


'The number of found communities is: 164861'


'The number of users in the largest community is: 26'


#### 10. Using Graph Algorithms answer **one** of the following questions:
    A. Which beer has the most similar reviews as the beer `Super Bock Stout`?
    B. Which user is the most influential when it comes to reviews made?

**B) Which user is the most influential when it comes to reviews made?**

In [18]:
# Step 0 - Dropping the existing graph named 'user_review_community' to ensure that it is possible to create a new graph
try:
    query = """
        CALL gds.graph.drop('user_review_community') YIELD graphName;
    """
    # Execute the Cypher query to drop the graph if it exists
    result = execute_read(driver, query)
except Exception as e:
    # If there is any error during the drop operation, print the error details
    pprint(e)


# Step 1 - Create a new graph projection for the PageRank algorithm
# This projection will include 'USER' and 'BEERS' as nodes, and the 'REVIEWED' relationships
# between 'REVIEWS' and 'BEERS' will be represented with a weight equal to the count of reviews
try:
    query_projection = """
        CALL gds.graph.project.cypher(
            'user_influence_graph',
            'MATCH (u:USER) RETURN id(u) AS id UNION ALL MATCH (b:BEERS) RETURN id(b) AS id',
            'MATCH (u:USER)<-[:POSTED]-(r:REVIEWS)<-[:REVIEWED]-(b:BEERS)
             RETURN id(u) AS source, id(b) AS target, count(r) AS weight'
        )
    """
    # Execute the query to project the graph using the provided node and relationship queries
    result = execute_read(driver, query_projection)
    # Pretty print the result, which will show details of the projected graph
    pprint(result)
    print('\n')
except Exception as e:
    # Print any exceptions that occur during graph projection
    pprint(e)


# Step 2 - Execute the PageRank algorithm on the projected graph to determine
# the influence of users based on their review connections
# The 'weight' property from the relationships is used by PageRank to assess influence
try:
    query_pagerank = """
        CALL gds.pageRank.stream('user_influence_graph', {relationshipWeightProperty: 'weight'})
        YIELD nodeId, score
        RETURN gds.util.asNode(nodeId).name AS UserName, score
        ORDER BY score DESC
        LIMIT 1
    """
    # Execute the PageRank algorithm, which computes the PageRank score
    # for each node and returns the score alongside the user's name
    pagerank_result = execute_read(driver, query_pagerank)
    # Pretty print the result, which will show the most influential user based on the PageRank score
    print('\n')
    print(f'The most influential user, when it comes to reviews made is: "{pagerank_result[0][0]}" with a PageRank score of {pagerank_result[0][1]}')
except Exception as e:
    # Print any exceptions that occur while running the PageRank algorithm
    pprint(e)

[<Record nodeQuery='MATCH (u:USER) RETURN id(u) AS id UNION ALL MATCH (b:BEERS) RETURN id(b) AS id' relationshipQuery='MATCH (u:USER)<-[:POSTED]-(r:REVIEWS)<-[:REVIEWED]-(b:BEERS)\n             RETURN id(u) AS source, id(b) AS target, count(r) AS weight' graphName='user_influence_graph' nodeCount=882681 relationshipCount=9071251 projectMillis=706356>]




The most influential user, when it comes to reviews made is: "Heady Topper" with a PageRank score of 193.41620036398234


#### 11. If you had to pick 3 beers to recommend using only this database, which would you pick and why?

<div style="border: 1px solid #ffdb4d; background-color: #ffffcc; color: #666600; padding: 10px; margin: 10px 0; border-radius: 5px;">
    <strong>Warning:</strong> You must have the libraries <code>ipywidgets</code>, <code>IPython</code>, and <code>jupyter-ui-poll</code> installed to run the following recommendation algorithm.<br>
    <strong>Warning:</strong> You have to run the following cell for the widgets to be displayed.
</div>

In [20]:
from ipywidgets import Dropdown, Button, VBox
from IPython.display import display
from jupyter_ui_poll import ui_events
from time import sleep

# Fetch unique country names from the database
res = execute_read(driver, 'MATCH (c:COUNTRIES) RETURN DISTINCT c.name')
countries = list(map(lambda x: x['c.name'], res))

# Define options for ranking the characteristics
rank_options = ['Look', 'Smell', 'Taste']
rank = {'Look': 0, 'Smell': 0, 'Taste': 0}

# Create dropdown widgets for country selection and characteristics ranking
cs = Dropdown(
        options=countries,
        value=None,
        disabled=False,
        description='Country: '
)

# Dropdown for first preference
rank1 = Dropdown(
    options=rank_options,
    value=None,
    disabled=False,
    description='1st: '
)  

# Dropdown for second preference
rank2 = Dropdown(
    options=rank_options,
    value=None,
    disabled=False,
    description='2nd: '
)  

# Dropdown for third preference
rank3 = Dropdown(
    options=rank_options,
    value=None,
    disabled=False,
    description='3rd: '
)


# Handler function for country dropdown changes
def dropdown_handler_cs(change):
    global dropdown_input_cs
    dropdown_input_cs = change.new


# Handler function for first preference dropdown changes
def dropdown_rank1(change):
    rank_options.remove(change.new)
    rank[change.new] = 3
    rank2.options = rank_options


# Handler function for second preference dropdown changes
def dropdown_rank2(change):
    rank_options.remove(change.new)
    rank[change.new] = 2
    rank3.options = rank_options


# Handler function for third preference dropdown changes
def dropdown_rank3(change):
    rank[change.new] = 1


# Attach handlers to the dropdown widgets
cs.observe(dropdown_handler_cs, names='value')
rank1.observe(dropdown_rank1, names='value')
rank2.observe(dropdown_rank2, names='value')
rank3.observe(dropdown_rank3, names='value')


# Display the dropdown widgets
print('                 Select your country:')
display(cs)
print('            Rank characteristics based on your preferences:')
display(rank1)
display(rank2)
display(rank3)


# Function to answer the question
def get_result():
    # Construct a Cypher query to recommend beers based on the user's preferences and selected country
    # This query computes a weighted score for each beer based on the user's ratings for look, smell, and taste
    # It only considers reviews that have non-null and numeric values for these characteristics
    cypher_query = f"""
    MATCH (cntr:COUNTRIES)<-[:IN]-(c:CITIES)<-[:IN]-(br:BREWERIES)-[:BREWED]->(b:BEERS)-[r:REVIEWED]->(review:REVIEWS)
    WHERE cntr.name = '{dropdown_input_cs}' AND review.look IS NOT NULL AND review.smell IS NOT NULL AND review.taste IS NOT NULL
    AND toFloat(review.look) IS NOT NULL AND toFloat(review.smell) IS NOT NULL AND toFloat(review.taste) IS NOT NULL
    WITH b, 
         AVG(toFloat(review.look)) * {rank['Look']} AS look_score, 
         AVG(toFloat(review.smell)) * {rank['Smell']} AS smell_score, 
         AVG(toFloat(review.taste)) * {rank['Taste']} AS taste_score
    WITH b, look_score + smell_score + taste_score AS weighted_score
    RETURN b.name AS Beer_Name, weighted_score
    ORDER BY weighted_score DESC
    LIMIT 3
    """
    
    # Execute the query and print the top 3 recommended beers based on the computed weighted scores
    result = execute_read(driver, cypher_query)
    print('\n')
    print(f"Beers from {dropdown_input_cs} recommended for you to try based on your preferences:")
    for i, beer in enumerate(result):
        print(f"\n\t{i+1}. {beer['Beer_Name']}")  # Printing each recommended beer's name


# Trigger result
button = Button(
    description='Search for beers',
    disabled=False
)
display(button)


# Flag to control the event polling loop
clicked = False
def on_button_clicked(b):
    global clicked
    clicked = True

button.on_click(on_button_clicked)


# Event loop that waits for the button click event before proceeding
with ui_events() as poll:
    while not clicked:
        poll(10) # poll queued UI events including button
        sleep(0.3) # wait for 0.3 seconds before checking again


# Once the button is clicked, call the function to display the result
get_result()

                 Select your country:


Dropdown(description='Country: ', options=('BE', 'US', 'GB', 'AU', 'IT', 'CA', 'GR', 'FR', 'AT', 'ES', 'RU', '…

            Rank characteristics based on your preferences:


Dropdown(description='1st: ', options=('Look', 'Smell', 'Taste'), value=None)

Dropdown(description='2nd: ', options=('Look', 'Smell', 'Taste'), value=None)

Dropdown(description='3rd: ', options=('Look', 'Smell', 'Taste'), value=None)

Button(description='Search for beers', style=ButtonStyle())



Beers from NL recommended for you to try based on your preferences:

	1. Binkie Claws 2018 Woodford BA

	2. Black Damnation Molen's 666 Edition

	3. Hel & Verdoemenis - MacAllan BA


**Justification**

The recommendation algorithm is designed to provide personalized beer suggestions based on individual user preferences concerning three key beer characteristics: look, smell, and taste. These attributes are widely recognized as significant factors affecting a person's enjoyment of beer.

Here is a detailed justification for the recommendation algorithm:

**Personalization through User Input:**
The algorithm starts by engaging with the user to gather personalized data. It prompts the user to select their country, ensuring that recommended beers are locally available. This geographical consideration narrows down the scope of the search to a relevant subset of beers.

**Weighted Characteristics:**
To capture the user's preference, the algorithm asks them to rank the importance of look, smell, and taste on a scale of 1 to 3, with 1 being the most important. This ranking is then inverted to serve as weights (with a higher ranking leading to a higher weight) to reflect the user's preference in the scoring system — the most cherished characteristic has the greatest influence on the recommendation.

**Aggregated Review Data:**
The algorithm leverages aggregated review data from other users to inform its suggestions. It computes a weighted average score for each beer based on historical review data, where the look, smell, and taste scores from reviews are multiplied by the user's corresponding weights. Only reviews with valid data are considered, ensuring reliability.

**Scoring and Ranking:**
Each beer is scored by summing the weighted average scores of look, smell, and taste. The final score represents an aggregate assessment of each beer's alignment with the user's stated preferences.

**Top Recommendations:**
The algorithm sorts all beers by their computed scores in descending order. The top three beers, which are the best match for the user's preferences, are then presented as recommendations. This approach offers a concise and focused selection for the user to consider.

**Adaptation to Diverse Preferences:**
By adjusting weights based on user input, the algorithm is flexible and easily adapts to a wide range of taste profiles. Whether a user values the aroma over appearance or taste over aroma, the recommendations will reflect these preferences.

**Streamlined User Experience:**
The algorithm's use of widgets and dropdowns for input collection provides a user-friendly interface. The direct and straightforward interaction does not overwhelm the user with choices but guides them through a simple process to collect essential preference data.

**Performance Considerations:**
The algorithm efficiently queries the database and processes data. By using Cypher queries tailored to perform calculations within the database engine, it avoids the overhead of transferring large data sets for external processing.

To sum up, the algorithm stands on the principles of personalization, data-driven insights, and user experience. It understands the user's unique taste, leverages community data for informed recommendations, and presents a friendly interface for interaction. The output is a set of three beers, each likely to please the user based on the intricate balance of look, smell, and taste that they have prioritized.